In [119]:
import pandas as pd
from keras.models import load_model
import numpy as np
from datetime import datetime, timedelta

class TickerReader(object):
    def __init__(self, filename):
        self.df = pd.read_csv(filename)
        self.df = self.df.drop(columns=['open', 'high', 'low', 'close', 'volume', 'dividend', 'split'])
        self.df = self.df.head(10000).sort_values(by=['time']).groupby(['id'])
        self.model = load_model('test.h5')
        
    def predict2(self):
        ids = []
        for i, df in list(self.df):
#             TODO: get row count
            x = df[['adjusted_open', 'adjusted_high', 'adjusted_low', 'adjusted_close', 'adjusted_volume', 'adjusted_open']].values
            if x.shape[0] >= 60:
#                 ids.append([df['id'], df['time']])
                ids.append([i, datetime.strptime(df['time'].iloc[-1], '%Y-%m-%d')])

        x_input = np.empty((len(ids), 60, 6), dtype=np.float32)

        current_point = 0

        for i, df in list(self.df):
            df['moving_average'] = df['adjusted_close'].rolling(5).mean()
            x = df[['adjusted_open', 'adjusted_high', 'adjusted_low', 'adjusted_close', 'adjusted_volume', 'moving_average']].values
            if x.shape[0] >= 60:
                x_input[current_point, :, :] = x[-60:, :]
            current_point += 1
            
        y = model.predict(x_input)
        return y

In [120]:
reader = TickerReader('/Users/eureyuri/Desktop/School/Brandeis/Computer-Science/149B/cosi-149b-project2/tmp.csv')

In [121]:
reader.predict2()

array([[ 0.08192189,  0.122981  ,  0.24014714,  0.2754598 , -0.4026155 ,
        -0.05310421]], dtype=float32)

In [118]:
for i, df in list(reader.df):
    x = df[['adjusted_open', 'adjusted_high', 'adjusted_low', 'adjusted_close', 'adjusted_volume', 'adjusted_open']].values
    if x.shape[0] >= 60:
        x = x[-60:, :]
        x = x.reshape((1, 60, 6))
        y = model.predict(x)
        print(y)

[[ 0.08192189  0.122981    0.24014714  0.2754598  -0.4026155  -0.05310421]]


In [44]:
(datetime.strptime('2019-01-01', '%Y-%m-%d') + timedelta(days=7)).strftime('%Y-%m-%d')

'2019-01-08'